# Python development notes

In [1]:
import numpy as np
from pydantic import BaseModel
from typing import List
from datetime import datetime
import logging
import collections

## Potential projects

* Audio dimensionality reduction. Find similar samples from a 2d visualization https://papers.nips.cc/paper/8634-learning-representations-for-time-series-clustering

# Python and Types

## Duck Typing

If it looks like a duck and quacks like a duck, then it's probably a duck.

## Pydantic

### Numpy arrays in Pydantic Model

In [2]:
class NumpyNDArray(np.ndarray):
    """Pydantic compatible data type for numpy ndarrays"""

    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def __modify_schema__(cls, field_schema):
        field_schema.update(type="array", items={"type": "number"})

    @classmethod
    def validate(cls, value: List) -> np.ndarray:
        # Transform input to ndarray
        return np.array(value)
    

class InputDataModel(BaseModel):

    date_time: datetime
    array: NumpyNDArray

    class Config:
        json_encoders = {
            np.ndarray: lambda v: v.tolist(),
            datetime: lambda v: datetime.strftime(v, "%Y-%m-%d %H:%M:%S"),
        }

# Classes

## Special Method Names

Classes can have many special method names that allow behaviour such as addition between objects or getting values by index. Comprehensive list: https://docs.python.org/3/reference/datamodel.html#special-method-names

## Inheritance

## Abstract Base Class

Let's you create a class that cannot be instantiated, only extended, and enforcing a certain interface.

In [8]:
from abc import ABC, abstractmethod
from typing import List

class Exchange(ABC):
    
    @abstractmethod
    def connect(self):
        pass
    
    @abstractmethod
    def get_market_data(self, stock: str) -> List[float]:
        pass

try:
    exchange = Exchange()
except TypeError as exc:
    print(f"ERROR: {exc}")
    
class AvanzaExchange(Exchange):
    
    def connect(self):
        print("Connecting to Avanza")
        
    def get_market_data(self, stock: str) -> List[float]:
        print(f"Getting data from {stock}")
        return [1,2,3,4,5]
    
exchange = AvanzaExchange()
exchange.connect()
exchange.get_market_data("GME")

ERROR: Can't instantiate abstract class Exchange with abstract methods connect, get_market_data
Connecting to Avanza
Getting data from GME


[1, 2, 3, 4, 5]

### Super

Let's you call methods from the parent class without specifying it by name

In [3]:
class LoggingDict(dict):
    def __setitem__(self, key, value):
        logging.info('Setting %r to %r' % (key, value))
        super().__setitem__(key, value)  # Call the __setitem__ function of parent class: dict

        
# Logging ordered dict
class LoggingOD(LoggingDict, collections.OrderedDict):
    pass

The ancestor tree for our new class is: `LoggingOD`, `LoggingDict`, `OrderedDict`, `dict`, `object`. For our purposes, the important result is that `OrderedDict` was inserted after `LoggingDict` and before `dict`! This means that the `super()` call in `LoggingDict.__setitem__` now dispatches the key/value update to `OrderedDict` instead of `dict`.

Three requirements:

1. the method being called by super() needs to exist
2. the caller and callee need to have a matching argument signature
3. and every occurrence of the method needs to use super()

To solve 1. in an MRO-chain we might have to add a `Root` class that we guarantee to be called before object. `Root` can also employ defensive programming to make sure it isn't masking another method later in the chain.

In [4]:
class Root:
    def draw(self):
        # the delegation chain stops here
        assert not hasattr(super(), 'draw')

class Shape(Root):
    def __init__(self, shapename, **kwds):
        self.shapename = shapename
        super().__init__(**kwds)
    def draw(self):
        print('Drawing.  Setting shape to:', self.shapename)
        super().draw()

class ColoredShape(Shape):
    def __init__(self, color, **kwds):
        self.color = color
        super().__init__(**kwds)
    def draw(self):
        print('Drawing.  Setting color to:', self.color)
        super().draw()

cs = ColoredShape(color='blue', shapename='square')
cs.draw()

Drawing.  Setting color to: blue
Drawing.  Setting shape to: square


Solve 2. by having classes accept keyword arguments and `**kwargs` and pass forward the unused `**kwargs`. `object.__init__` takes no arguments, so all arguments must be used by then.

In [5]:
class Shape:
    def __init__(self, shapename, **kwds):
        self.shapename = shapename
        super().__init__(**kwds)        

        
class ColoredShape(Shape):
    def __init__(self, color, **kwds):
        self.color = color
        super().__init__(**kwds)


cs = ColoredShape(color='red', shapename='circle')

Point 3. is easily solved if we are building the chain from scratch, simply by adding `super()`. But some external classes might not use `super()` or do not inherit from the root class. Then use an adapter class. The following example does note make `super()` calls and has an `__init__` that is incompatible with `object.__init__` and it does not inherit from `Root`:

In [8]:
class Moveable:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def draw(self):
        print('Drawing at position:', self.x, self.y)

If we want to use this class with our cooperatively designed ColoredShape hierarchy, we need to make an adapter with the requisite super() calls:

In [9]:
class MoveableAdapter(Root):
    def __init__(self, x, y, **kwds):
        self.movable = Moveable(x, y)
        super().__init__(**kwds)
    def draw(self):
        self.movable.draw()
        super().draw()

class MovableColoredShape(ColoredShape, MoveableAdapter):
    pass

MovableColoredShape(color='red', shapename='triangle',
                    x=10, y=20).draw()

Drawing at position: 10 20
